# Table of Contents
1. [Imports and definitions](#imports-and-definitions)
2. [Load data](#load-data)
3. [Prepare data](#prepare-data)
4. [Define objectives](#define-objectives)
5. [Start tuning](#start-tuning)
6. [Conclusion](#conclusion)

# Imports and definitions

In [1]:
from pathlib import Path
import time
import pickle

import polars as pl
import numpy as np

import numpy as np
import xgboost as xgb
from sklearn.metrics import f1_score

from sklearn.model_selection import StratifiedKFold, train_test_split

from sklearn.linear_model import RidgeClassifier, LogisticRegression
from sklearn.ensemble import StackingClassifier

import optuna

_ = pl.Config.set_tbl_cols(None)
_ = pl.Config.set_fmt_str_lengths(500)
_ = pl.Config.set_fmt_float("full")

In [2]:
import warnings
warnings.filterwarnings('ignore', category=RuntimeWarning, module='sklearn')
warnings.filterwarnings('ignore', module='lightgbm')

In [ ]:
base_dir = Path('/workspaces/data-scientist-at-magenta')
code_dir = base_dir / 'notebooks'
data_dir = code_dir / "data"
features_dir = data_dir / 'features'
train_dir = data_dir / 'train'
db_dir = 'sqlite:///data/models/{}.db'
artifacts_dir = data_dir / "models/artifacts"


In [4]:
# Helper function to load model artifacts
def load_model_artifact(path):
    with open(path, "rb") as f:
        return pickle.load(f)

# Load data

In [11]:
%%time

train = pl.read_parquet(train_dir / 'data-meta-v0-50.parquet')

CPU times: user 5.71 ms, sys: 15.8 ms, total: 21.5 ms
Wall time: 37.2 ms


# Prepare data

In [12]:
X = train.select(pl.exclude(['rating_account_id', 'customer_id', 'has_done_upselling']))
y = train.select('has_done_upselling')

In [13]:
X_meta, X_final_meta, y_meta, y_final_meta = train_test_split(
    X, y, test_size=0.5, random_state=42,
)

In [14]:
train_meta = pl.concat([X_meta, y_meta], how='horizontal')

In [15]:
data_data_b1 = train_meta.filter(pl.col("available_gb") < 25).drop('available_gb')
data_data_b2 = train_meta.filter(pl.col("available_gb") < 25).drop('available_gb')

X_train_data_b1 = data_data_b1.select(pl.exclude(['rating_account_id', 'customer_id', 'has_done_upselling']))
X_train_data_b2 = data_data_b2.select(pl.exclude(['rating_account_id', 'customer_id', 'has_done_upselling']))

y_train_data_b1 = data_data_b1.select('has_done_upselling')
y_train_data_b2 = data_data_b2.select('has_done_upselling')


In [16]:
data_days_b1 = train_meta.filter(pl.col("contract_lifetime_days") < 1000).drop('contract_lifetime_days')
data_days_b2 = train_meta.filter(pl.col("contract_lifetime_days") >= 1000).drop('contract_lifetime_days')

X_train_days_b1 = data_days_b1.select(pl.exclude(['rating_account_id', 'customer_id', 'has_done_upselling']))
X_train_days_b2 = data_days_b2.select(pl.exclude(['rating_account_id', 'customer_id', 'has_done_upselling']))

y_train_days_b1 = data_days_b1.select('has_done_upselling')
y_train_days_b2 = data_days_b2.select('has_done_upselling')


In [17]:
data_age_b1 = train_meta.filter(pl.col("age") < 55).drop('age')
data_age_b2 = train_meta.filter(pl.col("age") >= 55).drop('age')

X_train_age_b1 = data_age_b1.select(pl.exclude(['rating_account_id', 'customer_id', 'has_done_upselling']))
X_train_age_b2 = data_age_b2.select(pl.exclude(['rating_account_id', 'customer_id', 'has_done_upselling']))

y_train_age_b1 = data_age_b1.select('has_done_upselling')
y_train_age_b2 = data_age_b2.select('has_done_upselling')


In [18]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Define objectives

In [27]:
def stacking_objective(trial, X, y, skf, base_models):
    # Define Ridge parameters to tune
    ridge_params = {
        'alpha': trial.suggest_float('alpha', 1e-4, 100.0, log=True),
        'solver': trial.suggest_categorical('solver', ['auto', 'svd', 'cholesky', 'lsqr']),
        'class_weight': trial.suggest_categorical('class_weight', [None, 'balanced'])
    }
    
    # Create stacking classifier with Ridge meta-learner
    stacking_clf = StackingClassifier(
        estimators=base_models,  
        final_estimator=RidgeClassifier(**ridge_params),
        cv='prefit',  # No need for the base models to be trained
        stack_method='predict_proba'
    )
    
    cv_scores = []
    optimal_thresholds = []
    
    # Use the same cross-validation approach as base models
    for train_idx, valid_idx in skf.split(X, y):
        # Split data
        train_x, valid_x = X[train_idx], X[valid_idx] 
        train_y, valid_y = y[train_idx], y[valid_idx]
        
        # Train stacking classifier
        stacking_clf.fit(train_x.to_numpy(), train_y.to_numpy().ravel())
        
        # Get probabilities
        preds = stacking_clf.predict(valid_x.to_numpy())
        
        # Find optimal threshold using your existing function
        f1 = f1_score(valid_y.to_numpy().ravel(), preds)
        
        cv_scores.append(f1)
    
    return np.mean(cv_scores)

# Start tuning

In [ ]:
n_trials = 50

## Age

In [19]:
# Prepare base models for B1 and B2 separately (age)
base_models_age_b1 = []
base_models_age_b2 = []

# XGBoost B1 (age)
xgb_clf_age_b1 = load_model_artifact(artifacts_dir / "pre_xgb_age_b1.pkl")
# base_models_age_b1.append(('xgb_age_b1', xgb_clf_age_b1))

# XGBoost B2 (age)
xgb_clf_age_b2 = load_model_artifact(artifacts_dir / "pre_xgb_age_b2.pkl")
base_models_age_b2.append(('xgb_age_b2', xgb_clf_age_b2))

# Random Forest B1 (age)
rf_clf_age_b1 = load_model_artifact(artifacts_dir / "pre_rf_age_b1.pkl")
base_models_age_b1.append(('rf_age_b1', rf_clf_age_b1))

# Random Forest B2 (age)
rf_clf_age_b2 = load_model_artifact(artifacts_dir / "pre_rf_age_b2.pkl")
base_models_age_b2.append(('rf_age_b2', rf_clf_age_b2))

# LightGBM B1 (age)
lgb_clf_age_b1 = load_model_artifact(artifacts_dir / "pre_lgb_age_b1.pkl")
# base_models_age_b1.append(('lgb_age_b1', lgb_clf_age_b1))

# LightGBM B2 (age)
lgb_clf_age_b2 = load_model_artifact(artifacts_dir / "pre_lgb_age_b2.pkl")
# base_models_age_b2.append(('lgb_age_b2', lgb_clf_age_b2))

# CatBoost B1 (age)
cat_clf_age_b1 = load_model_artifact(artifacts_dir / "pre_cat_age_b1.pkl")
base_models_age_b1.append(('cat_age_b1', cat_clf_age_b1))

# CatBoost B2 (age)
cat_clf_age_b2 = load_model_artifact(artifacts_dir / "pre_cat_age_b2.pkl")
# base_models_age_b2.append(('cat_age_b2', cat_clf_age_b2))

# HistGradientBoosting B1 (age)
histgb_clf_age_b1 = load_model_artifact(artifacts_dir / "pre_histgb_age_b1.pkl")
base_models_age_b1.append(('histgb_age_b1', histgb_clf_age_b1))

# HistGradientBoosting B2 (age)
histgb_clf_age_b2 = load_model_artifact(artifacts_dir / "pre_histgb_age_b2.pkl")
base_models_age_b2.append(('histgb_age_b2', histgb_clf_age_b2))


In [65]:
age_b1_meta_study = optuna.create_study(
    study_name="meta_ridge_age_b1_optimization",
    direction="maximize",
    storage=db_dir.format('meta_learners_study'),
    load_if_exists=True
)
age_b1_meta_study.optimize(lambda trial: stacking_objective(trial, X_train_age_b1, y_train_age_b1, skf, base_models_age_b1), n_trials=n_trials)
print(f"\nBest Stacking score: {age_b1_meta_study.best_value}")
print(f"Best Stacking params: {age_b1_meta_study.best_params}")

[I 2025-07-02 09:45:01,087] Using an existing study with name 'meta_ridge_age_b1_optimization' instead of creating a new one.
[I 2025-07-02 09:45:01,840] Trial 202 finished with value: 0.150461467774079 and parameters: {'alpha': 0.019272157857400753, 'solver': 'lsqr', 'class_weight': 'balanced'}. Best is trial 152 with value: 0.17275778398665537.
[I 2025-07-02 09:45:02,505] Trial 203 finished with value: 0.15009002305991098 and parameters: {'alpha': 0.053158517709567, 'solver': 'lsqr', 'class_weight': 'balanced'}. Best is trial 152 with value: 0.17275778398665537.
[I 2025-07-02 09:45:03,133] Trial 204 finished with value: 0.1503646022305179 and parameters: {'alpha': 0.03129721662046393, 'solver': 'lsqr', 'class_weight': 'balanced'}. Best is trial 152 with value: 0.17275778398665537.
[I 2025-07-02 09:45:03,861] Trial 205 finished with value: 0.15039755157033988 and parameters: {'alpha': 0.024086769276383816, 'solver': 'lsqr', 'class_weight': 'balanced'}. Best is trial 152 with value: 0.


Best Stacking score: 0.17275778398665537
Best Stacking params: {'alpha': 0.02944866210721213, 'solver': 'auto', 'class_weight': 'balanced'}


In [58]:
age_b2_meta_study = optuna.create_study(
    study_name="meta_ridge_age_b2_optimization",
    direction="maximize",
    storage=db_dir.format('meta_learners_study'),
    load_if_exists=True
)
# model: Ridge meta-stacking on age_b2, using X_train_age_b2/y_train_age_b2 and base_models_age_b2
age_b2_meta_study.optimize(lambda trial: stacking_objective(trial, X_train_age_b2, y_train_age_b2, skf, base_models_age_b2), n_trials=n_trials)
print(f"\nBest Stacking score: {age_b2_meta_study.best_value}")
print(f"Best Stacking params: {age_b2_meta_study.best_params}")

[I 2025-07-02 09:39:25,400] Using an existing study with name 'meta_ridge_age_b2_optimization' instead of creating a new one.
[I 2025-07-02 09:39:27,881] Trial 154 finished with value: 0.1338473184009092 and parameters: {'alpha': 6.494589439966312, 'solver': 'lsqr', 'class_weight': 'balanced'}. Best is trial 152 with value: 0.16895526150137577.
[I 2025-07-02 09:39:30,362] Trial 155 finished with value: 0.1338473184009092 and parameters: {'alpha': 5.072191838784726, 'solver': 'lsqr', 'class_weight': 'balanced'}. Best is trial 152 with value: 0.16895526150137577.
[I 2025-07-02 09:39:32,912] Trial 156 finished with value: 0.0 and parameters: {'alpha': 11.4459957573685, 'solver': 'lsqr', 'class_weight': None}. Best is trial 152 with value: 0.16895526150137577.
[I 2025-07-02 09:39:35,444] Trial 157 finished with value: 0.1338473184009092 and parameters: {'alpha': 7.398333619302283, 'solver': 'svd', 'class_weight': 'balanced'}. Best is trial 152 with value: 0.16895526150137577.
[I 2025-07-02


Best Stacking score: 0.16895526150137577
Best Stacking params: {'alpha': 9.092803857312077, 'solver': 'lsqr', 'class_weight': 'balanced'}


---

## Contract

In [20]:
# Prepare base models for B1 and B2 separately (days)
base_models_days_b1 = []
base_models_days_b2 = []

# XGBoost B1 (days)
xgb_clf_days_b1 = load_model_artifact(artifacts_dir / "pre_xgb_days_b1.pkl")
# base_models_days_b1.append(('xgb_days_b1', xgb_clf_days_b1))

# XGBoost B2 (days)
xgb_clf_days_b2 = load_model_artifact(artifacts_dir / "pre_xgb_days_b2.pkl")
# base_models_days_b2.append(('xgb_days_b2', xgb_clf_days_b2))

# Random Forest B1 (days)
rf_clf_days_b1 = load_model_artifact(artifacts_dir / "pre_rf_days_b1.pkl")
base_models_days_b1.append(('rf_days_b1', rf_clf_days_b1))

# Random Forest B2 (days)
rf_clf_days_b2 = load_model_artifact(artifacts_dir / "pre_rf_days_b2.pkl")
base_models_days_b2.append(('rf_days_b2', rf_clf_days_b2))

# LightGBM B1 (days)
lgb_clf_days_b1 = load_model_artifact(artifacts_dir / "pre_lgb_days_b1.pkl")
base_models_days_b1.append(('lgb_days_b1', lgb_clf_days_b1))

# LightGBM B2 (days)
lgb_clf_days_b2 = load_model_artifact(artifacts_dir / "pre_lgb_days_b2.pkl")
base_models_days_b2.append(('lgb_days_b2', lgb_clf_days_b2))

# CatBoost B1 (days)
cat_clf_days_b1 = load_model_artifact(artifacts_dir / "pre_cat_days_b1.pkl")
base_models_days_b1.append(('cat_days_b1', cat_clf_days_b1))

# CatBoost B2 (days)
cat_clf_days_b2 = load_model_artifact(artifacts_dir / "pre_cat_days_b2.pkl")
base_models_days_b2.append(('cat_days_b2', cat_clf_days_b2))

# HistGradientBoosting B1 (days)
histgb_clf_days_b1 = load_model_artifact(artifacts_dir / "pre_histgb_days_b1.pkl")
# base_models_days_b1.append(('histgb_days_b1', histgb_clf_days_b1))

# HistGradientBoosting B2 (days)
histgb_clf_days_b2 = load_model_artifact(artifacts_dir / "pre_histgb_days_b2.pkl")
# base_models_days_b2.append(('histgb_days_b2', histgb_clf_days_b2))


In [21]:
days_b1_meta_study = optuna.create_study(
    study_name="meta_ridge_age_b1_optimization",
    direction="maximize",
    storage=db_dir.format('meta_learners_study'),
    load_if_exists=True
)
days_b1_meta_study.optimize(lambda trial: stacking_objective(trial, X_train_days_b1, y_train_days_b1, skf, base_models_days_b1), n_trials=n_trials)
print(f"\nBest Stacking score: {days_b1_meta_study.best_value}")
print(f"Best Stacking params: {days_b1_meta_study.best_params}")

[I 2025-07-02 11:31:00,589] Using an existing study with name 'meta_ridge_age_b1_optimization' instead of creating a new one.


NameError: name 'n_trials' is not defined

In [61]:
days_b2_meta_study = optuna.create_study(
    study_name="meta_ridge_age_b2_optimization",
    direction="maximize",
    storage=db_dir.format('meta_learners_study'),
    load_if_exists=True
)
days_b2_meta_study.optimize(lambda trial: stacking_objective(trial, X_train_days_b2, y_train_days_b2, skf, base_models_days_b2), n_trials=n_trials)
print(f"\nBest Stacking score: {days_b2_meta_study.best_value}")
print(f"Best Stacking params: {days_b2_meta_study.best_params}")

[I 2025-07-02 09:41:41,930] Using an existing study with name 'meta_ridge_age_b2_optimization' instead of creating a new one.
/Users/danlab/code/magenta-task/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/danlab/code/magenta-task/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/danlab/code/magenta-task/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/danlab/code/magenta-task/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users

[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6
[LightGBM] [Warning] feature_fraction is set=0.8162034579535279, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8162034579535279
[LightGBM] [Warning] bagging_fraction is set=0.9276862426700836, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9276862426700836
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6
[LightGBM] [Warning] feature_fraction is set=0.8162034579535279, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8162034579535279
[LightGBM] [Warning] bagging_fraction is set=0.9276862426700836, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9276862426700836
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6
[LightGBM] [Warning] feature_fraction is set=0.8162034579535279, colsample

/Users/danlab/code/magenta-task/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/danlab/code/magenta-task/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/danlab/code/magenta-task/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/danlab/code/magenta-task/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
[I 2025-07-02 09:41:42,317] Trial 204 finished with value: 0.1638634058805549 and parameters: {'alpha': 8.03723855555765, 'solver': 

[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6
[LightGBM] [Warning] feature_fraction is set=0.8162034579535279, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8162034579535279
[LightGBM] [Warning] bagging_fraction is set=0.9276862426700836, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9276862426700836
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6
[LightGBM] [Warning] feature_fraction is set=0.8162034579535279, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8162034579535279
[LightGBM] [Warning] bagging_fraction is set=0.9276862426700836, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9276862426700836
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6
[LightGBM] [Warning] feature_fraction is set=0.8162034579535279, colsample

/Users/danlab/code/magenta-task/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/danlab/code/magenta-task/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/danlab/code/magenta-task/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/danlab/code/magenta-task/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/danlab/code/magenta-task/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have va

...

[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6
[LightGBM] [Warning] feature_fraction is set=0.8162034579535279, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8162034579535279
[LightGBM] [Warning] bagging_fraction is set=0.9276862426700836, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9276862426700836
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6
[LightGBM] [Warning] feature_fraction is set=0.8162034579535279, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8162034579535279
[LightGBM] [Warning] bagging_fraction is set=0.9276862426700836, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9276862426700836
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6
[LightGBM] [Warning] feature_fraction is set=0.8162034579535279, colsample

/Users/danlab/code/magenta-task/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/danlab/code/magenta-task/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/danlab/code/magenta-task/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/danlab/code/magenta-task/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/danlab/code/magenta-task/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have va

[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6
[LightGBM] [Warning] feature_fraction is set=0.8162034579535279, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8162034579535279
[LightGBM] [Warning] bagging_fraction is set=0.9276862426700836, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9276862426700836
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6
[LightGBM] [Warning] feature_fraction is set=0.8162034579535279, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8162034579535279
[LightGBM] [Warning] bagging_fraction is set=0.9276862426700836, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9276862426700836
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6
[LightGBM] [Warning] feature_fraction is set=0.8162034579535279, colsample

/Users/danlab/code/magenta-task/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/danlab/code/magenta-task/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/danlab/code/magenta-task/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/danlab/code/magenta-task/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
[I 2025-07-02 09:42:00,994] Trial 253 finished with value: 0.16640282019401337 and parameters: {'alpha': 28.229314902726117, 'solver

[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6
[LightGBM] [Warning] feature_fraction is set=0.8162034579535279, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8162034579535279
[LightGBM] [Warning] bagging_fraction is set=0.9276862426700836, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9276862426700836
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6
[LightGBM] [Warning] feature_fraction is set=0.8162034579535279, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8162034579535279
[LightGBM] [Warning] bagging_fraction is set=0.9276862426700836, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9276862426700836
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6
[LightGBM] [Warning] feature_fraction is set=0.8162034579535279, colsample

---


## Data

In [22]:
# Prepare base models for B1 and B2 separately (data)
base_models_data_b1 = []
base_models_data_b2 = []

# XGBoost B1 (data)
xgb_clf_data_b1 = load_model_artifact(artifacts_dir / "pre_xgb_data_b1.pkl")
base_models_data_b1.append(('xgb_data_b1', xgb_clf_data_b1))

# XGBoost B2 (data)
xgb_clf_data_b2 = load_model_artifact(artifacts_dir / "pre_xgb_data_b2.pkl")
base_models_data_b2.append(('xgb_data_b2', xgb_clf_data_b2))

# Random Forest B1 (data)
rf_clf_data_b1 = load_model_artifact(artifacts_dir / "pre_rf_data_b1.pkl")
# base_models_data_b1.append(('rf_data_b1', rf_clf_data_b1))

# Random Forest B2 (data)
rf_clf_data_b2 = load_model_artifact(artifacts_dir / "pre_rf_data_b2.pkl")
# base_models_data_b2.append(('rf_data_b2', rf_clf_data_b2))

# LightGBM B1 (data)
lgb_clf_data_b1 = load_model_artifact(artifacts_dir / "pre_lgb_data_b1.pkl")
# base_models_data_b1.append(('lgb_data_b1', lgb_clf_data_b1))

# LightGBM B2 (data)
lgb_clf_data_b2 = load_model_artifact(artifacts_dir / "pre_lgb_data_b2.pkl")
# base_models_data_b2.append(('lgb_data_b2', lgb_clf_data_b2))

# CatBoost B1 (data)
cat_clf_data_b1 = load_model_artifact(artifacts_dir / "pre_cat_data_b1.pkl")
base_models_data_b1.append(('cat_data_b1', cat_clf_data_b1))

# CatBoost B2 (data)
cat_clf_data_b2 = load_model_artifact(artifacts_dir / "pre_cat_data_b2.pkl")
base_models_data_b2.append(('cat_data_b2', cat_clf_data_b2))

# HistGradientBoosting B1 (data)
histgb_clf_data_b1 = load_model_artifact(artifacts_dir / "pre_histgb_data_b1.pkl")
# base_models_data_b1.append(('histgb_data_b1', histgb_clf_data_b1))

# HistGradientBoosting B2 (data)
histgb_clf_data_b2 = load_model_artifact(artifacts_dir / "pre_histgb_data_b2.pkl")
# base_models_data_b2.append(('histgb_data_b2', histgb_clf_data_b2))


In [63]:
data_b1_meta_study = optuna.create_study(
    study_name="meta_ridge_age_b1_optimization",
    direction="maximize",
    storage=db_dir.format('meta_learners_study'),
    load_if_exists=True
)
data_b1_meta_study.optimize(lambda trial: stacking_objective(trial, X_train_data_b1, y_train_data_b1, skf, base_models_data_b1), n_trials=n_trials)
print(f"\nBest Stacking score: {data_b1_meta_study.best_value}")
print(f"Best Stacking params: {data_b1_meta_study.best_params}")

[I 2025-07-02 09:42:01,072] Using an existing study with name 'meta_ridge_age_b1_optimization' instead of creating a new one.
[I 2025-07-02 09:42:01,181] Trial 152 finished with value: 0.17275778398665537 and parameters: {'alpha': 0.02944866210721213, 'solver': 'auto', 'class_weight': 'balanced'}. Best is trial 152 with value: 0.17275778398665537.
[I 2025-07-02 09:42:01,255] Trial 153 finished with value: 0.17275778398665537 and parameters: {'alpha': 0.023904274181823105, 'solver': 'auto', 'class_weight': 'balanced'}. Best is trial 152 with value: 0.17275778398665537.
[I 2025-07-02 09:42:01,328] Trial 154 finished with value: 0.17275778398665537 and parameters: {'alpha': 0.0325062168170535, 'solver': 'auto', 'class_weight': 'balanced'}. Best is trial 152 with value: 0.17275778398665537.
[I 2025-07-02 09:42:01,464] Trial 155 finished with value: 0.17275778398665537 and parameters: {'alpha': 0.02682816786018081, 'solver': 'auto', 'class_weight': 'balanced'}. Best is trial 152 with value:


Best Stacking score: 0.17275778398665537
Best Stacking params: {'alpha': 0.02944866210721213, 'solver': 'auto', 'class_weight': 'balanced'}


In [64]:
data_b2_meta_study = optuna.create_study(
    study_name="meta_ridge_age_b2_optimization",
    direction="maximize",
    storage=db_dir.format('meta_learners_study'),
    load_if_exists=True
)
data_b2_meta_study.optimize(lambda trial: stacking_objective(trial, X_train_data_b2, y_train_data_b2, skf, base_models_data_b2), n_trials=n_trials)
print(f"\nBest Stacking score: {data_b2_meta_study.best_value}")
print(f"Best Stacking params: {data_b2_meta_study.best_params}")

[I 2025-07-02 09:42:04,968] Using an existing study with name 'meta_ridge_age_b2_optimization' instead of creating a new one.
[I 2025-07-02 09:42:05,094] Trial 254 finished with value: 0.1652602633821382 and parameters: {'alpha': 35.36148647859042, 'solver': 'lsqr', 'class_weight': 'balanced'}. Best is trial 152 with value: 0.16895526150137577.
[I 2025-07-02 09:42:05,181] Trial 255 finished with value: 0.16644046874478918 and parameters: {'alpha': 30.339382782760882, 'solver': 'lsqr', 'class_weight': 'balanced'}. Best is trial 152 with value: 0.16895526150137577.
[I 2025-07-02 09:42:05,269] Trial 256 finished with value: 0.1632369154883585 and parameters: {'alpha': 45.79977555738733, 'solver': 'lsqr', 'class_weight': 'balanced'}. Best is trial 152 with value: 0.16895526150137577.
[I 2025-07-02 09:42:05,359] Trial 257 finished with value: 0.16644046874478918 and parameters: {'alpha': 30.28631049496538, 'solver': 'lsqr', 'class_weight': 'balanced'}. Best is trial 152 with value: 0.168955


Best Stacking score: 0.16906387212859242
Best Stacking params: {'alpha': 25.70489492011668, 'solver': 'auto', 'class_weight': 'balanced'}


---

## Final decision

In [23]:
def create_bins_split_features(X, y, split_configs, base_models, seed=42):
    """
    Create bins split features based on different column thresholds.
    For each row in X, get predictions from all split models and combine them.
    
    Args:
        X (pl.DataFrame): Training features
        y (pl.DataFrame): Training labels
        split_configs (list): List of dicts with keys: 'name', 'column', 'threshold'
        base_models (dict): Dict mapping split names to tuples of (model_b1, model_b2)
        seed (int): Random seed for shuffling
    
    Returns:
        pl.DataFrame: DataFrame with predictions from each split as columns
    """
    predictions_dict = {}
    
    for config in split_configs:
        name = config['name']
        column = config['column']
        threshold = config['threshold']
        
        # Get the models for this split
        model_b1, model_b2 = base_models[name]
        
        # Initialize predictions array for this split
        predictions = np.zeros(X.height)
        
        # For each row, determine which model to use based on the threshold
        column_values = X.select(column).to_numpy().ravel()
        b1_mask = column_values < threshold
        b2_mask = ~b1_mask
        
        # Prepare features without the split column for prediction
        X_for_prediction = X.drop(column).with_row_index("idx")
        
        # Process b1 split (rows where column < threshold)
        if np.any(b1_mask):
            b1_indices = np.where(b1_mask)[0]
            b1_X = X_for_prediction.filter(pl.col("idx").is_in(b1_indices)).drop("idx")
            b1_predictions = model_b1.predict(b1_X.to_numpy())
            predictions[b1_mask] = b1_predictions
        
        # Process b2 split (rows where column >= threshold)
        if np.any(b2_mask):
            b2_indices = np.where(b2_mask)[0]
            b2_X = X_for_prediction.filter(pl.col("idx").is_in(b2_indices)).drop("idx")
            b2_predictions = model_b2.predict(b2_X.to_numpy())
            predictions[b2_mask] = b2_predictions
        
        predictions_dict[name] = predictions
    
    # Create final DataFrame with all predictions as columns plus labels
    predictions_dict['label'] = y.to_numpy().ravel()
    final_df = pl.DataFrame(predictions_dict)
    
    # Shuffle the results
    final_df = final_df.sample(fraction=1.0, with_replacement=False, seed=seed)
    
    return final_df

In [34]:
def final_meta_objective(trial, X, y, skf,
                         split_configs, stacking_models):
    """
    Objective function for optimizing the final meta-learner that takes decisions
    based on predictions from all stacking models (age, days, data splits)
    Args:
        trial: Optuna trial object
        X: Original features including 'age' column
        y: Target variable
        skf: StratifiedKFold object
        stacking_age_b1: StackingClassifier for age < 55
        stacking_age_b2: StackingClassifier for age >= 55
        stacking_days_b1: StackingClassifier for contract_lifetime_days < 1000
        stacking_days_b2: StackingClassifier for contract_lifetime_days >= 1000
        stacking_data_b1: StackingClassifier for available_gb < 25
        stacking_data_b2: StackingClassifier for available_gb >= 25
    """

    ridge_params = {
        'alpha': trial.suggest_float('alpha', 1e-4, 100.0, log=True),
        'solver': trial.suggest_categorical('solver', ['auto', 'svd', 'cholesky', 'lsqr']),
        'class_weight': trial.suggest_categorical('class_weight', [None, 'balanced'])
    }
    final_meta_clf = RidgeClassifier(**ridge_params)

    cv_scores = []

    for train_idx, valid_idx in skf.split(X, y):
        train_x, valid_x = X[train_idx], X[valid_idx]
        train_y, valid_y = y[train_idx], y[valid_idx]


        meta__train_X_y = create_bins_split_features(train_x, train_y, split_configs, stacking_models)
        final_meta_clf.fit(meta__train_X_y.select(list(stacking_models.keys())).to_numpy(), meta__train_X_y.select('label').to_numpy().ravel())

        meta__val_X_y = create_bins_split_features(valid_x, valid_y, split_configs, stacking_models)
        preds = final_meta_clf.predict(meta__val_X_y.select(list(stacking_models.keys())).to_numpy())
        f1 = f1_score(meta__val_X_y.select('label').to_numpy().ravel(), preds)
        cv_scores.append(f1)

    return np.mean(cv_scores)


In [25]:
# Helper to extract best params and create RidgeClassifier
def ridge_from_study_path(study_path, study_name):
    study = optuna.load_study(study_name=study_name, storage=study_path)
    params = study.best_params.copy()
    valid_keys = {'alpha', 'solver', 'class_weight'}
    params = {k: v for k, v in params.items() if k in valid_keys}
    return RidgeClassifier(**params)

# Age B1 stacking classifier
stacking_age_b1 = StackingClassifier(
    estimators=base_models_age_b1,
    final_estimator=ridge_from_study_path(db_dir.format('meta_learners_study'), "meta_ridge_age_b1_optimization"),
    cv='prefit',
    stack_method='predict_proba'
)

# Age B2 stacking classifier
stacking_age_b2 = StackingClassifier(
    estimators=base_models_age_b2,
    final_estimator=ridge_from_study_path(db_dir.format('meta_learners_study'), "meta_ridge_age_b2_optimization"),
    cv='prefit',
    stack_method='predict_proba'
)

# Days B1 stacking classifier
stacking_days_b1 = StackingClassifier(
    estimators=base_models_days_b1,
    final_estimator=ridge_from_study_path(db_dir.format('meta_learners_study'), "meta_ridge_age_b1_optimization"),
    cv='prefit',
    stack_method='predict_proba'
)

# Days B2 stacking classifier
stacking_days_b2 = StackingClassifier(
    estimators=base_models_days_b2,
    final_estimator=ridge_from_study_path(db_dir.format('meta_learners_study'), "meta_ridge_age_b2_optimization"),
    cv='prefit',
    stack_method='predict_proba'
)

# Data B1 stacking classifier
stacking_data_b1 = StackingClassifier(
    estimators=base_models_data_b1,
    final_estimator=ridge_from_study_path(db_dir.format('meta_learners_study'), "meta_ridge_age_b1_optimization"),
    cv='prefit',
    stack_method='predict_proba'
)

# Data B2 stacking classifier
stacking_data_b2 = StackingClassifier(
    estimators=base_models_data_b2,
    final_estimator=ridge_from_study_path(db_dir.format('meta_learners_study'), "meta_ridge_age_b2_optimization"),
    cv='prefit',
    stack_method='predict_proba'
)

In [26]:
# Fit stacking models for each split
stacking_age_b1.fit(X_train_age_b1.to_numpy(), y_train_age_b1.to_numpy().ravel())
stacking_age_b2.fit(X_train_age_b2.to_numpy(), y_train_age_b2.to_numpy().ravel())

stacking_days_b1.fit(X_train_days_b1.to_numpy(), y_train_days_b1.to_numpy().ravel())
stacking_days_b2.fit(X_train_days_b2.to_numpy(), y_train_days_b2.to_numpy().ravel())

stacking_data_b1.fit(X_train_data_b1.to_numpy(), y_train_data_b1.to_numpy().ravel())
stacking_data_b2.fit(X_train_data_b2.to_numpy(), y_train_data_b2.to_numpy().ravel())


[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
[LightGBM] [Warning] feature_fraction is set=0.8151925596377104, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8151925596377104
[LightGBM] [Warning] bagging_fraction is set=0.9704150221294421, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9704150221294421
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6
[LightGBM] [Warning] feature_fraction is set=0.8162034579535279, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8162034579535279
[LightGBM] [Warning] bagging_fraction is set=0.9276862426700836, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9276862426700836


/Users/danlab/code/magenta-task/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/danlab/code/magenta-task/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


,estimators,"[('xgb_data_b2', ...), ('cat_data_b2', ...)]"
,final_estimator,RidgeClassifi...ht='balanced')
,cv,'prefit'
,stack_method,'predict_proba'
,n_jobs,None
,passthrough,False
,verbose,0
,objective,'binary:logistic'
,base_score,None
,booster,'gbtree'
,callbacks,None


In [ ]:
split_configs = [
    {'name': 'age', 'column': 'age', 'threshold': 55},
    {'name': 'days', 'column': 'contract_lifetime_days', 'threshold': 1000}, 
    {'name': 'data', 'column': 'available_gb', 'threshold': 25}
]

stacking_models = {
    'age': (stacking_age_b1, stacking_age_b2),
    'days': (stacking_days_b1, stacking_days_b2),
    'data': (stacking_data_b1, stacking_data_b2)
}

final_meta_study = optuna.create_study(
    study_name="meta_ridge_final_decision_optimization",
    direction="maximize",
    storage=db_dir.format('meta_learners_study'),
    load_if_exists=True
)

final_meta_study.optimize(
    lambda trial: final_meta_objective(
        trial,
        X_final_meta,
        y_final_meta,
        skf,
        split_configs,
        stacking_models
    ),
    n_trials=n_trials
)

print(f"\nBest Final Meta Stacking score: {final_meta_study.best_value}")
print(f"Best Final Meta Stacking params: {final_meta_study.best_params}")

[I 2025-07-02 10:34:29,946] Using an existing study with name 'meta_ridge_final_decision_optimization' instead of creating a new one.
/Users/danlab/code/magenta-task/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/danlab/code/magenta-task/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
[LightGBM] [Warning] feature_fraction is set=0.8151925596377104, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8151925596377104
[LightGBM] [Warning] bagging_fraction is set=0.9704150221294421, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9704150221294421
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6
[LightGBM] [Warning] feature_fraction is set=0.8162034579535279, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8162034579535279
[LightGBM] [Warning] bagging_fraction is set=0.9276862426700836, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9276862426700836


/Users/danlab/code/magenta-task/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/danlab/code/magenta-task/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
[LightGBM] [Warning] feature_fraction is set=0.8151925596377104, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8151925596377104
[LightGBM] [Warning] bagging_fraction is set=0.9704150221294421, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9704150221294421
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6
[LightGBM] [Warning] feature_fraction is set=0.8162034579535279, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8162034579535279
[LightGBM] [Warning] bagging_fraction is set=0.9276862426700836, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9276862426700836


/Users/danlab/code/magenta-task/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/danlab/code/magenta-task/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


...

[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
[LightGBM] [Warning] feature_fraction is set=0.8151925596377104, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8151925596377104
[LightGBM] [Warning] bagging_fraction is set=0.9704150221294421, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9704150221294421
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6
[LightGBM] [Warning] feature_fraction is set=0.8162034579535279, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8162034579535279
[LightGBM] [Warning] bagging_fraction is set=0.9276862426700836, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9276862426700836


/Users/danlab/code/magenta-task/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/danlab/code/magenta-task/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
[LightGBM] [Warning] feature_fraction is set=0.8151925596377104, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8151925596377104
[LightGBM] [Warning] bagging_fraction is set=0.9704150221294421, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9704150221294421
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6
[LightGBM] [Warning] feature_fraction is set=0.8162034579535279, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8162034579535279
[LightGBM] [Warning] bagging_fraction is set=0.9276862426700836, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9276862426700836


/Users/danlab/code/magenta-task/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/danlab/code/magenta-task/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
[I 2025-07-02 10:40:30,797] Trial 54 finished with value: 0.27347061357085434 and parameters: {'alpha': 1.717763142980173, 'solver': 'lsqr', 'class_weight': 'balanced'}. Best is trial 5 with value: 0.27347061357085434.


[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
[LightGBM] [Warning] feature_fraction is set=0.8151925596377104, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8151925596377104
[LightGBM] [Warning] bagging_fraction is set=0.9704150221294421, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9704150221294421
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6
[LightGBM] [Warning] feature_fraction is set=0.8162034579535279, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8162034579535279
[LightGBM] [Warning] bagging_fraction is set=0.9276862426700836, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9276862426700836

Best Final Meta Stacking score: 0.27347061357085434
Best Final Meta Stacking params: {'alpha': 3.537691648211495, 'solver': 'svd', 'class_weight': 'balanced'}


In [8]:
import itertools

# List of all stacking classifier variable names
stacking_clf_names = [
    "stacking_age", "stacking_days", 'stacking_data'
]

# Get all possible non-empty combinations
all_combinations = []
for r in range(2, len(stacking_clf_names)):
    combos = list(itertools.combinations(stacking_clf_names, r))
    all_combinations.extend(combos)

# Print all combinations
for combo in all_combinations:
    print(combo)

('stacking_age', 'stacking_days')
('stacking_age', 'stacking_data')
('stacking_days', 'stacking_data')


In [35]:
split_configs = [
    {'name': 'age', 'column': 'age', 'threshold': 55},
    {'name': 'days', 'column': 'contract_lifetime_days', 'threshold': 1000}, 
]

stacking_models = {
    'age': (stacking_age_b1, stacking_age_b2),
    'days': (stacking_days_b1, stacking_days_b2),
}

final_meta_study = optuna.create_study(
    study_name="meta_ridge_final_decision_age_days_optimization",
    direction="maximize",
    storage=db_dir.format('meta_learners_study'),
    load_if_exists=True
)

final_meta_study.optimize(
    lambda trial: final_meta_objective(
        trial,
        X_final_meta,
        y_final_meta,
        skf,
        split_configs,
        stacking_models
    ),
    n_trials=n_trials
)

print(f"\nBest Final Meta Stacking score: {final_meta_study.best_value}")
print(f"Best Final Meta Stacking params: {final_meta_study.best_params}")

[I 2025-07-02 11:35:57,301] Using an existing study with name 'meta_ridge_final_decision_age_days_optimization' instead of creating a new one.
/Users/danlab/code/magenta-task/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/danlab/code/magenta-task/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
[LightGBM] [Warning] feature_fraction is set=0.8151925596377104, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8151925596377104
[LightGBM] [Warning] bagging_fraction is set=0.9704150221294421, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9704150221294421
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6
[LightGBM] [Warning] feature_fraction is set=0.8162034579535279, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8162034579535279
[LightGBM] [Warning] bagging_fraction is set=0.9276862426700836, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9276862426700836


/Users/danlab/code/magenta-task/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/danlab/code/magenta-task/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
[LightGBM] [Warning] feature_fraction is set=0.8151925596377104, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8151925596377104
[LightGBM] [Warning] bagging_fraction is set=0.9704150221294421, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9704150221294421
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6
[LightGBM] [Warning] feature_fraction is set=0.8162034579535279, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8162034579535279
[LightGBM] [Warning] bagging_fraction is set=0.9276862426700836, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9276862426700836


/Users/danlab/code/magenta-task/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/danlab/code/magenta-task/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


...

[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
[LightGBM] [Warning] feature_fraction is set=0.8151925596377104, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8151925596377104
[LightGBM] [Warning] bagging_fraction is set=0.9704150221294421, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9704150221294421
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6
[LightGBM] [Warning] feature_fraction is set=0.8162034579535279, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8162034579535279
[LightGBM] [Warning] bagging_fraction is set=0.9276862426700836, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9276862426700836


/Users/danlab/code/magenta-task/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/danlab/code/magenta-task/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
[LightGBM] [Warning] feature_fraction is set=0.8151925596377104, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8151925596377104
[LightGBM] [Warning] bagging_fraction is set=0.9704150221294421, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9704150221294421
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6
[LightGBM] [Warning] feature_fraction is set=0.8162034579535279, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8162034579535279
[LightGBM] [Warning] bagging_fraction is set=0.9276862426700836, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9276862426700836


/Users/danlab/code/magenta-task/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/danlab/code/magenta-task/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
[I 2025-07-02 11:38:32,638] Trial 52 finished with value: 0.1521012440464255 and parameters: {'alpha': 0.057340249631567676, 'solver': 'auto', 'class_weight': 'balanced'}. Best is trial 6 with value: 0.1521012440464255.


[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
[LightGBM] [Warning] feature_fraction is set=0.8151925596377104, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8151925596377104
[LightGBM] [Warning] bagging_fraction is set=0.9704150221294421, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9704150221294421
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6
[LightGBM] [Warning] feature_fraction is set=0.8162034579535279, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8162034579535279
[LightGBM] [Warning] bagging_fraction is set=0.9276862426700836, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9276862426700836

Best Final Meta Stacking score: 0.1521012440464255
Best Final Meta Stacking params: {'alpha': 0.10053237676225904, 'solver': 'auto', 'class_weight': 'balanced'}


In [36]:
split_configs = [
    {'name': 'age', 'column': 'age', 'threshold': 55},
    {'name': 'data', 'column': 'available_gb', 'threshold': 25}
]

stacking_models = {
    'age': (stacking_age_b1, stacking_age_b2),
    'data': (stacking_data_b1, stacking_data_b2)
}

final_meta_study = optuna.create_study(
    study_name="meta_ridge_final_decision_age_data_optimization",
    direction="maximize",
    storage=db_dir.format('meta_learners_study'),
    load_if_exists=True
)

final_meta_study.optimize(
    lambda trial: final_meta_objective(
        trial,
        X_final_meta,
        y_final_meta,
        skf,
        split_configs,
        stacking_models
    ),
    n_trials=n_trials
)

print(f"\nBest Final Meta Stacking score: {final_meta_study.best_value}")
print(f"Best Final Meta Stacking params: {final_meta_study.best_params}")

[I 2025-07-02 11:38:32,770] Using an existing study with name 'meta_ridge_final_decision_age_data_optimization' instead of creating a new one.
[I 2025-07-02 11:38:35,539] Trial 0 finished with value: 0.1394265483656798 and parameters: {'alpha': 0.0005507046751878807, 'solver': 'auto', 'class_weight': 'balanced'}. Best is trial 0 with value: 0.1394265483656798.
[I 2025-07-02 11:38:38,251] Trial 1 finished with value: 0.1394265483656798 and parameters: {'alpha': 0.001633500867050798, 'solver': 'cholesky', 'class_weight': 'balanced'}. Best is trial 0 with value: 0.1394265483656798.
[I 2025-07-02 11:38:41,007] Trial 2 finished with value: 0.1394265483656798 and parameters: {'alpha': 0.0042856173063518235, 'solver': 'auto', 'class_weight': 'balanced'}. Best is trial 0 with value: 0.1394265483656798.
[I 2025-07-02 11:38:43,788] Trial 3 finished with value: 0.0 and parameters: {'alpha': 0.21112435325756743, 'solver': 'cholesky', 'class_weight': None}. Best is trial 0 with value: 0.13942654836


Best Final Meta Stacking score: 0.1394265483656798
Best Final Meta Stacking params: {'alpha': 0.0005507046751878807, 'solver': 'auto', 'class_weight': 'balanced'}


In [39]:
split_configs = [
    {'name': 'days', 'column': 'contract_lifetime_days', 'threshold': 1000}, 
    {'name': 'data', 'column': 'available_gb', 'threshold': 25}
]

stacking_models = {
    'days': (stacking_days_b1, stacking_days_b2),
    'data': (stacking_data_b1, stacking_data_b2)
}

final_meta_study = optuna.create_study(
    study_name="meta_ridge_final_decision_days_data_optimization",
    direction="maximize",
    storage=db_dir.format('meta_learners_study'),
    load_if_exists=True
)

final_meta_study.optimize(
    lambda trial: final_meta_objective(
        trial,
        X_final_meta,
        y_final_meta,
        skf,
        split_configs,
        stacking_models
    ),
    n_trials=n_trials
)

print(f"\nBest Final Meta Stacking score: {final_meta_study.best_value}")
print(f"Best Final Meta Stacking params: {final_meta_study.best_params}")

[I 2025-07-02 11:57:35,997] Using an existing study with name 'meta_ridge_final_decision_days_data_optimization' instead of creating a new one.
/Users/danlab/code/magenta-task/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/danlab/code/magenta-task/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/danlab/code/magenta-task/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/danlab/code/magenta-task/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  war

[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
[LightGBM] [Warning] feature_fraction is set=0.8151925596377104, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8151925596377104
[LightGBM] [Warning] bagging_fraction is set=0.9704150221294421, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9704150221294421
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6
[LightGBM] [Warning] feature_fraction is set=0.8162034579535279, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8162034579535279
[LightGBM] [Warning] bagging_fraction is set=0.9276862426700836, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9276862426700836
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
[LightGBM] [Warning] feature_fraction is set=0.8151925596377104, colsample

/Users/danlab/code/magenta-task/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/danlab/code/magenta-task/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/danlab/code/magenta-task/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/danlab/code/magenta-task/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/danlab/code/magenta-task/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have va

[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
[LightGBM] [Warning] feature_fraction is set=0.8151925596377104, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8151925596377104
[LightGBM] [Warning] bagging_fraction is set=0.9704150221294421, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9704150221294421
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6
[LightGBM] [Warning] feature_fraction is set=0.8162034579535279, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8162034579535279
[LightGBM] [Warning] bagging_fraction is set=0.9276862426700836, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9276862426700836
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
[LightGBM] [Warning] feature_fraction is set=0.8151925596377104, colsample

/Users/danlab/code/magenta-task/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/danlab/code/magenta-task/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/danlab/code/magenta-task/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/danlab/code/magenta-task/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/danlab/code/magenta-task/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have va

...

[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6
[LightGBM] [Warning] feature_fraction is set=0.8162034579535279, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8162034579535279
[LightGBM] [Warning] bagging_fraction is set=0.9276862426700836, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9276862426700836
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
[LightGBM] [Warning] feature_fraction is set=0.8151925596377104, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8151925596377104
[LightGBM] [Warning] bagging_fraction is set=0.9704150221294421, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9704150221294421
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6
[LightGBM] [Warning] feature_fraction is set=0.8162034579535279, colsample

/Users/danlab/code/magenta-task/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/danlab/code/magenta-task/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/danlab/code/magenta-task/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/danlab/code/magenta-task/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/danlab/code/magenta-task/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have va

[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6
[LightGBM] [Warning] feature_fraction is set=0.8162034579535279, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8162034579535279
[LightGBM] [Warning] bagging_fraction is set=0.9276862426700836, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9276862426700836
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
[LightGBM] [Warning] feature_fraction is set=0.8151925596377104, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8151925596377104
[LightGBM] [Warning] bagging_fraction is set=0.9704150221294421, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9704150221294421
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6
[LightGBM] [Warning] feature_fraction is set=0.8162034579535279, colsample

/Users/danlab/code/magenta-task/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/danlab/code/magenta-task/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/danlab/code/magenta-task/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/danlab/code/magenta-task/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/danlab/code/magenta-task/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have va

[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6
[LightGBM] [Warning] feature_fraction is set=0.8162034579535279, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8162034579535279
[LightGBM] [Warning] bagging_fraction is set=0.9276862426700836, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9276862426700836
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
[LightGBM] [Warning] feature_fraction is set=0.8151925596377104, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8151925596377104
[LightGBM] [Warning] bagging_fraction is set=0.9704150221294421, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9704150221294421
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6
[LightGBM] [Warning] feature_fraction is set=0.8162034579535279, colsample

In [41]:
def print_study_results(name, study):
    print(f"{name}:")
    print(f"  Best score: {study.best_value}")
    print(f"  Best params: {study.best_params}\n")

storage = db_dir.format('meta_learners_study')

age_b1_meta_study = optuna.load_study(study_name="meta_ridge_age_b1_optimization", storage=storage)
age_b2_meta_study = optuna.load_study(study_name="meta_ridge_age_b2_optimization", storage=storage)
days_b1_meta_study = optuna.load_study(study_name="meta_ridge_age_b1_optimization", storage=storage)
days_b2_meta_study = optuna.load_study(study_name="meta_ridge_age_b2_optimization", storage=storage)
data_b1_meta_study = optuna.load_study(study_name="meta_ridge_age_b1_optimization", storage=storage)
data_b2_meta_study = optuna.load_study(study_name="meta_ridge_age_b2_optimization", storage=storage)
final_meta_study = optuna.load_study(study_name="meta_ridge_final_decision_optimization", storage=storage)
final_meta_age_days_study = optuna.load_study(study_name="meta_ridge_final_decision_age_days_optimization", storage=storage)
final_meta_age_data_study = optuna.load_study(study_name="meta_ridge_final_decision_age_data_optimization", storage=storage)
final_meta_days_data_study = optuna.load_study(study_name="meta_ridge_final_decision_days_data_optimization", storage=storage)


print_study_results("Age B1 Meta Study", age_b1_meta_study)
print_study_results("Age B2 Meta Study", age_b2_meta_study)
print_study_results("Days B1 Meta Study", days_b1_meta_study)
print_study_results("Days B2 Meta Study", days_b2_meta_study)
print_study_results("Data B1 Meta Study", data_b1_meta_study)
print_study_results("Data B2 Meta Study", data_b2_meta_study)
print_study_results("Final Meta Study", final_meta_study)
print_study_results("Final Meta Age-Days Study", final_meta_age_days_study)
print_study_results("Final Meta Age-DataStudy", final_meta_age_data_study)
print_study_results("Final Meta Days-Data Study", final_meta_days_data_study)


Age B1 Meta Study:
  Best score: 0.17275778398665537
  Best params: {'alpha': 0.02944866210721213, 'solver': 'auto', 'class_weight': 'balanced'}

Age B2 Meta Study:
  Best score: 0.16906387212859242
  Best params: {'alpha': 25.70489492011668, 'solver': 'auto', 'class_weight': 'balanced'}

Days B1 Meta Study:
  Best score: 0.17275778398665537
  Best params: {'alpha': 0.02944866210721213, 'solver': 'auto', 'class_weight': 'balanced'}

Days B2 Meta Study:
  Best score: 0.16906387212859242
  Best params: {'alpha': 25.70489492011668, 'solver': 'auto', 'class_weight': 'balanced'}

Data B1 Meta Study:
  Best score: 0.17275778398665537
  Best params: {'alpha': 0.02944866210721213, 'solver': 'auto', 'class_weight': 'balanced'}

Data B2 Meta Study:
  Best score: 0.16906387212859242
  Best params: {'alpha': 25.70489492011668, 'solver': 'auto', 'class_weight': 'balanced'}

Final Meta Study:
  Best score: 0.27347061357085434
  Best params: {'alpha': 3.537691648211495, 'solver': 'svd', 'class_weight